# Inspection du dataset
Affichage du CSV sous forme de DataFrame + tailles des DataLoaders.

In [2]:
import sys
from pathlib import Path
import pandas as pd
from data.dataset import create_dataloaders

In [3]:
# Paramètres
CSV_FILE = "../data/raw/prepared_dataset.csv"
BATCH_SIZE = 32

# Afficher le DataFrame (10 premières lignes)
df = pd.read_csv(CSV_FILE)
df.head(10)

,image_id,non_seg,seg,label
0,AIX034925,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN
1,AIX043024,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN
2,AIX055881,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN
3,ALF032718,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN
4,ALF032721,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN
5,ALF032741,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN
6,ALF032746,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN
7,ALF035277,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN
8,ALF035285,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN
9,ALF035290,/data/Downloads/Data_Projet_Complet/Data_Proje...,/data/Downloads/Data_Projet_Complet/Data_Proje...,NaN


In [4]:
# Créer les dataloaders
train_loader, val_loader = create_dataloaders(
    csv_file=CSV_FILE,
    batch_size=BATCH_SIZE,
    train_split=0.8,
    img_size=224,
    seed=42
)

print(f"Train set: {len(train_loader.dataset)} images")
print(f"Val set: {len(val_loader.dataset)} images")

Train set: 718 images
Val set: 180 images


In [5]:
import itertools
import pandas as pd

# Extraire les 10 premiers échantillons
data = []
for i, (img_non_seg, img_seg, label) in enumerate(itertools.islice(train_loader.dataset, 10)):
    data.append({
        'Sample': i,
        'Label': label.item() if hasattr(label, 'item') else label,
        'Non-Seg Shape': str(img_non_seg.shape),
        'Seg Shape': str(img_seg.shape)
    })

# Créer et afficher le DataFrame
samples_df = pd.DataFrame(data)
samples_df

print(f"Train set: {len(train_loader.dataset)} images")
print(f"Val set: {len(val_loader.dataset)} images")

Train set: 718 images
Val set: 180 images


In [6]:
import itertools
import pandas as pd

# Extraire les 10 premiers échantillons
data = []
for i, (img_non_seg, img_seg, label) in enumerate(itertools.islice(train_loader.dataset, 10)):
    data.append({
        'Sample': i,
        'Label': label.item() if hasattr(label, 'item') else label,
        'Non-Seg Shape': str(img_non_seg.shape),
        'Seg Shape': str(img_seg.shape)
    })

# Créer et afficher le DataFrame
samples_df = pd.DataFrame(data)
samples_df

,Sample,Label,Non-Seg Shape,Seg Shape
0,0,0.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"
1,1,1.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"
2,2,0.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"
3,3,1.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"
4,4,0.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"
5,5,1.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"
6,6,1.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"
7,7,1.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"
8,8,1.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"
9,9,0.0,"torch.Size([3, 224, 224])","torch.Size([3, 224, 224])"


In [7]:
import torch
import timm

# Import pour enregistrer les modèles CrossViT
import models.crossvit

from training.training import train_model_crossvit

/home/cytech/Ing3/DeepLearning/PROJET_DEEPL/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Paramètres d'entraînement (test rapide)
LABEL_COL = "label"
num_classes = df[LABEL_COL].nunique()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model("crossvit_tiny_224", pretrained=False, num_classes=num_classes).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
dataloaders = {"train": train_loader, "val": val_loader}

# Test d'entraînement (1 epoch)
model, history = train_model_crossvit(
    model=model,
    dataloaders=dataloaders,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    config="A",
    device=device,
    num_epochs=1
)

history


Epoch 1/1 [Config A]
----------


TRAIN:   0%|          | 0/23 [00:00<?, ?it/s]/home/cytech/Ing3/DeepLearning/PROJET_DEEPL/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
TRAIN:   0%|          | 0/23 [01:40<?, ?it/s]


KeyboardInterrupt: 